In [5]:
%load_ext autoreload
%reload_ext autoreload
%autoreload 2

import pandas as pd
import datetime
from imgw_measurements.measurements import Measurements, Measurement
from imgw_measurements.station_weather import Station
from imgw_measurements.stations import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Weather conditions measurements

In [65]:
main_measures = pd.DataFrame(Measurements.build_main_measurements())
main_measures

,measurement_code,measurement_en,measurement_pl,measurement_unit
0,B400E00200,Air Pressure,Ciśnienie atmosferyczne obserwowane-synop,hPa
1,B100E00200,Temperature,Temperatura powietrza chwilowa-synop,C
2,B500E00200,Humidity,Wilgotność względna chwilowa-synop,%
3,B201E002C1,Wind direction avg,Kierunek wiatru średni za ostatnie 10 min.-synop,st. kąta
4,B200E002C1,Wind speed avg,Prędkość wiatru średnia za ostatnie 10 min.-synop,m/s
5,B600B008FD,Rainfall,Suma opadu - deszcz-doba-synop,mm
6,B690E002FG,Sunshine 1h,Czas trwania usłonecznienia za ostatnią godzin...,minuta
7,B690B00800,Sunshine,Suma usłonecznienia-doba-synop,godzina
8,B650B008FD,Snowfall 2,Suma opadu - śnieg-doba-synop,mm
9,B610B00400,Visibility,Widzialność pozioma-synop,NaN


# Fetching data for a specific weather station

In [7]:
station = Station("KRAKÓW-BALICE")
(station.name, station.id)

('KRAKÓW-BALICE', 350190566)

In [99]:
# station = Station("KRAKÓW-OBSERWATORIUM")
# (station.name, station.id)

('KRAKÓW-OBSERWATORIUM', 250190390)

In [ ]:
cities = ['Warszawa', 'Wrocław', 'Gdańsk', 'Szczecin', 'Poznań', 'Katowice', 'Suwałki', 'Zakopane']

In [ ]:
get_city_stations("KRAKÓW")

In [46]:
krakow_stations_names = [
 'KRAKÓW-BALICE',
]
krakow_stations = [ Station(station_name) for station_name in krakow_stations_names]

In [49]:
START_DATE = datetime.date(2003, 1, 1)
END_DATE = datetime.date(2016, 12, 1)
missing_periods = {}
for station in krakow_stations:
    missing_periods[station] = station.fetch_data_for_period(START_DATE, END_DATE)

In [72]:
# csv_file = station.fetch_weather_conditions(2016, 5)
csv_file = './input_data/imgw/data/station_350190566_KRAKÓW-BALICE/2016/350190566_2016_04.csv.gz'
column_names = ['index', 'measurement_time', 'measurement_code', 'measurement_value']
conditions_in_month = pd.read_csv(csv_file, delimiter=';', header=None, names=column_names)


# Normalizing weather conditions dataframe

In [10]:
def format_measurements_dataframe(df, measurement_name_pl):
    output_df = df[df.measurement_pl == measurement_name_pl].copy()
    measurement = Measurement(name=measurement_name_pl)
    measurement_name_en = "{} [{}]".format(Measurements.MAIN_MEASURES[measurement_name_pl], measurement.unit)
    output_df.rename(columns={'measurement_value': measurement_name_en}, inplace=True)
    output_df[measurement_name_en] = pd.to_numeric(output_df[measurement_name_en], errors='coerce')
    columns_order = [ 'measurement_time', measurement_name_en ]
    output_df = output_df[columns_order]
    output_df.measurement_time = pd.to_datetime(output_df.measurement_time)
    output_df.set_index('measurement_time', inplace=True)
    return output_df 

def merge_all_measuremets(weather_df):
    weather_df = weather_df.merge(main_measures)
    all_measures_df = pd.DataFrame([])
    for measure_pl in Measurements.MAIN_MEASURES.keys():
        measure_en = Measurements.MAIN_MEASURES[measure_pl]
        measures_df = format_measurements_dataframe(weather_df, measure_pl)
        all_measures_df = all_measures_df.merge(measures_df, right_index=True, left_index=True, how='outer')
    all_measures_df['Wind speed avg [km/h]'] = all_measures_df['Wind speed avg [m/s]'] * 3.6
    all_measures_df = all_measures_df.resample('H').last()
    return all_measures_df.rename(columns={"Visibility [nan]": 'Visibility [m]'})


In [73]:
weather_conditions = merge_all_measuremets(conditions_in_month)

In [74]:
weather_conditions.columns

Index(['Air Pressure [hPa]', 'Temperature [C]', 'Humidity [%]',
       'Wind direction avg [st. kąta]', 'Wind speed avg [m/s]',
       'Rainfall [mm]', 'Sunshine 1h [minuta]', 'Sunshine [godzina]',
       'Snowfall 2 [mm]', 'Visibility [m]', 'Wind speed avg [km/h]'],
      dtype='object')

In [75]:
weather_conditions['Sunshine [godzina]'].sum()

0.0

In [ ]:
all_measures['Rainfall 1h [mm]'].min()

In [ ]:
all_measures['2016-02-27']